# Train locally

Train an MLflow Project directly on your machine using local compute and AzureML SDK.

## Table of Contents
1. Prerequisites
    - 1.1 Initialize Tracking Store and Experiment
    - 1.2 Configure the Backend Configuration object
    - 1.3 Modify your Environment specification
2. Submit Run
    - 2.1 User-managed environment
    - 2.2 System-managed environment


# Prerequisites 
Ensure you have done the following before running this notebook,
- Connected to an AML Workspace
- Your local conda environment has the necessary packages needed to run this project

## Initialize Tracking Store and Experiment

### Set the Tracking Store
Set the MLflow tracking URI to point to your Azure ML Workspace. The subsequent logging calls from MLflow APIs will go to Azure ML services and will be tracked under your Workspace.

In [ ]:
from azureml import core
from azureml.core import Workspace

import mlflow

workspace = Workspace.from_config()  
mlflow.set_tracking_uri(workspace.get_mlflow_tracking_uri())

### Create Experiment
Create an Mlflow Experiment to organize your runs. It can be set either by passing the name as a parameter in the mlflow.projects.run call or by the following,

In [ ]:
experiment_name = "mlflow-example"
mlflow.set_experiment(experiment_name)

## Create the Backend Configuration Object

The backend configuration object will store necesary information for the integration such as the compute target and whether to use your local managed environment or a system managed environment. 

The integration will accept "COMPUTE" and "USE_CONDA" as parameters where "COMPUTE" is set to the name of a remote target (not applicable for this training example) and "USE_CONDA" which creates a new environment for the project from the environment configuration file. You must set this to "False" and include it in the backend configuration object if you want to use your local environment for the project run. Mlflow accepts a dictionary object or a JSON file.

In [ ]:
# dictionary
backend_config = {"USE_CONDA": False}


## Add the Integration to your Environment Configuration

Add the azureml-mlflow package as a pip dependency to your environment configuration file. The project can run without this addition, but key artifacts and metrics will not be logged to your Workspace. Adding it to to the file will look like this,

```
name: mlflow-example
channels:
  - defaults
  - anaconda
  - conda-forge
dependencies:
  - python=3.6
  - scikit-learn=0.19.1
  - pip
  - pip:
    - mlflow
    - azureml-mlflow
```

# User Managed environment
For using your local conda environment, set `use_conda = False`. Ensure your local environment has all the necessary packages for running the project and you are specifying the **backend parameter** in any run call to be **"azureml"**.

In [ ]:
local_env_run = mlflow.projects.run(uri=".", 
                                    parameters={"alpha":0.3},
                                    backend = "azureml",
                                    backend_config = backend_config)


Note: All these calculations were run on your local machine, in the conda environment you defined above. You can find the results in:
- Your AzureML Experiments (a link to the run will be provided in the console)
- ~/.azureml/envs/azureml_xxxx for the conda environment you just created
- ~/AppData/Local/Temp/azureml_runs/train-on-local_xxxx for the machine learning models you trained (this path may differ depending on the platform you use). This folder also contains
    - Logs (under azureml_logs/)
    - Output pickled files (under outputs/)
    - The configuration files (credentials, local and docker image setups)
    - The train.py and mylib.py scripts
    - The current notebook

# System Mananged Environment

Now, instead of managing the setup of the environment yourself, you can ask the system to build a new conda environment for you using the environment configuration file in this project. If a backend configuration object is not provided in the call, the integration will default to creating a new conda environment. The environment is built once, and will be reused in subsequent executions as long as the conda dependencies remain unchanged.


In [ ]:
backend_config = {"USE_CONDA": True}
local_mlproject_run = mlflow.projects.run(uri=".", 
                                    parameters={"alpha":0.3},
                                    backend = "azureml",
                                    backend_config = backend_config)